# Scrape streaming tweets

In [1]:
import json
import tweepy
import dataset
import pandas as pd

from textblob import TextBlob
from sqlalchemy.exc import ProgrammingError

In [2]:
TRACK_TERMS = ["fast food","obesity"]
CSV_NAME = "tweets.csv"
TABLE_NAME = "obesity"

consumer_key = 'HjkdnggpHfCjmzKyeVK91u6st'
consumer_secret = 'gxDU30gPA9tS1RCGOfZKAvIt232WkSiNgThwbB7BH99rSU3WmQ'
access_token = '1455534963181465611-pXezqCKISRrCx06ePa1F40fX7BpRqj'
access_token_secret = 'W5PYhCV7Fm4EfqeoT0MlfFFL6imcsAeZ9ujyZUuy1nCHu'

In [3]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
# Creating an API object
api = tweepy.API(auth)

In [4]:
db = dataset.connect('sqlite:///tweets.db')

In [5]:
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        if status.retweeted:
            return

        description = status.user.description
        loc = status.user.location
        text = status.text
        coords = status.coordinates
        geo = status.geo
        name = status.user.screen_name
        user_created = status.user.created_at
        followers = status.user.followers_count
        id_str = status.id_str
        created = status.created_at
        retweets = status.retweet_count
        bg_color = status.user.profile_background_color
        blob = TextBlob(text)
        sent = blob.sentiment

        if geo is not None:
            geo = json.dumps(geo)

        if coords is not None:
            coords = json.dumps(coords)

        table = db[TABLE_NAME]
        try:
            table.insert(dict(
                user_description=description,
                user_location=loc,
                coordinates=coords,
                text=text,
                geo=geo,
                user_name=name,
                user_created=user_created,
                user_followers=followers,
                id_str=id_str,
                created=created,
                retweet_count=retweets,
                user_bg_color=bg_color,
                polarity=sent.polarity,
                subjectivity=sent.subjectivity,
            ))
        except ProgrammingError as err:
            print(err)

    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False

In [6]:
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(track=TRACK_TERMS)

ProtocolError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))